## Define the Objective of the Analysis

...

Analisi del sentiment relativo al dibattito sulla demolizione dello stadio di San Siro a Milano e individuazione di eventuali 'influencer' sul tema.

## Collect Data

In [1]:
# imports
import numpy as np
import pandas as pd
import tweepy
tweepy.__version__

import re
import string
import itertools
from collections import Counter
from datetime import datetime
from numpy.core.multiarray import result_type
import time

'4.8.0'

In [2]:
# insert the keys here
consumer_key = 'VPzjkqKl2y1uSTJQvnVqS9e1X' 
consumer_secret = 'STG2IzVMf65vPGeOvBQyzdeoKBExAr5sIkhOaBeDe2fnIN14vY'
access_token = '1508409949835214853-HIyZJ3oT32TijKsdNDhGFZEEQTWwau'
access_token_secret = 'uLcs9hUYmLdocxkaSfXo69Gii46TISu5qZj5F6f6fBfnW'

#### Tweets Download

The next step is creating an OAuthHandler instance. We pass our consumer key and access token which we defined above.

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

Next, we pass the OAuthHandler instance into the API method.

In [4]:
api = tweepy.API(auth)

Tweets that contain a specific hashtag

In [9]:
from tqdm.notebook import tqdm

import time
hashtag = '("#smartworking" OR "#remotework" OR "#lavoroagile")'

list_tweets = []

for i in range(10):
    for tweet in tweepy.Cursor(api.search_tweets, q=hashtag, count=50, lang='it').items(850):
      print('entering')
      full_text = api.get_status(tweet.id, tweet_mode='extended')._json['full_text']
      print(tweet.id)
      list_tweets.append([tweet.created_at, tweet.id, full_text, tweet.favorite_count, tweet.retweet_count, tweet.user.screen_name,
      tweet.user.location, tweet.retweeted, tweet.entities['user_mentions'], tweet.entities['hashtags']])
    for j in range(60):
        time.sleep(1)

# items is the maximum number of tweets to download.
# count is the number of tweets to return per page, up to a maximum of 100.

entering
1608467861609287680
entering
1608466273205374978
entering
1608465798594695168
entering
1608465280056922113
entering
1608464910668926976
entering
1608457676136284160
entering
1608453254978879490
entering
1608451832803921920
entering
1608450121355739137
entering
1608443757774667778
entering
1608443728368390144
entering
1608443480476454918
entering
1608441687139037184
entering
1608439990685691904
entering
1608438022038761472
entering
1608436212116918272
entering
1608436196878737410
entering
1608433820155654145
entering
1608429602816376836
entering
1608429439226122241
entering
1608429205850849282
entering
1608428894121779202
entering
1608427144463482880
entering
1608427098267418625
entering
1608426805756674048
entering
1608425274932174849
entering
1608425048507052033
entering
1608423989818580994
entering
1608422264479059968
entering
1608416916565069825
entering
1608416513656262656
entering
1608414157107302401
entering
1608413901896712192
entering
1608413367881940995
entering
16084

TooManyRequests: 429 Too Many Requests
88 - Rate limit exceeded

In [10]:
tweets = pd.DataFrame(list_tweets, columns=['date','id','text','like','n_rt','author','location','retweeted','user_mentions','hastags'])
tweets.head()

,date,id,text,like,n_rt,author,location,retweeted,user_mentions,hastags
0,2022-12-29 14:19:56+00:00,1608467861609287680,RT @JackRossano: La #Meloni per la #famiglia (...,0,15,atlantidelibri,castel san pietro Bo,False,"[{'screen_name': 'JackRossano', 'name': 'Jack ...","[{'text': 'Meloni', 'indices': [20, 27]}, {'te..."
1,2022-12-29 14:13:38+00:00,1608466273205374978,RT @ValePieraccini: Siamo indietro secoli luce...,0,2,saragra98354743,,False,"[{'screen_name': 'ValePieraccini', 'name': 'Va...","[{'text': 'smartworking', 'indices': [48, 61]}]"
2,2022-12-29 14:11:44+00:00,1608465798594695168,RT @lucague46760617: #smartworking @fattoquoti...,0,3,saragra98354743,,False,"[{'screen_name': 'lucague46760617', 'name': 'l...","[{'text': 'smartworking', 'indices': [21, 34]}]"
3,2022-12-29 14:09:41+00:00,1608465280056922113,RT @Ernesto23710724: @AlbertoBagnai Forse deve...,0,1,saragra98354743,,False,"[{'screen_name': 'Ernesto23710724', 'name': 'E...","[{'text': 'smartworking', 'indices': [89, 102]}]"
4,2022-12-29 14:08:13+00:00,1608464910668926976,RT @LucaGalizia: Chi già preconizza (con mezzo...,0,1,NicoladelConte8,,False,"[{'screen_name': 'LucaGalizia', 'name': 'Luca ...","[{'text': 'lockdown', 'indices': [68, 77]}, {'..."


In [11]:
len(list_tweets)

900

In [12]:
# Turn list_tweet into a DataFrame changing column names
tweets = pd.DataFrame(list_tweets, columns=['date','id','text','like','n_rt','author','location','retweeted','user_mentions','hastags'])
tweets.to_csv('../data/SW.csv')

In [ ]:
tweets_loaded = pd.read_csv('../data/SW.csv')
#tweets_loaded = tweets.drop('Unnamed: 0', axis=1)
tweets_loaded.head()

In [13]:
tweets.shape

(900, 10)

In [14]:
tweets.columns

Index(['date', 'id', 'text', 'like', 'n_rt', 'author', 'location', 'retweeted',
       'user_mentions', 'hastags'],
      dtype='object')

#### Data Pre-Processing

In [15]:
# Remove duplicates
tweets.drop_duplicates(subset ="id", inplace = True)
tweets.reset_index(drop = True, inplace = True)
tweets.shape

(850, 10)

In [16]:
# Change date format
day = tweets['date'].dt.day
month = tweets['date'].dt.month
year = tweets['date'].dt.year

date = year.astype(str) + month.astype(str).str.zfill(2) + day.astype(str).str.zfill(2)
date = pd.to_datetime(date, format='%Y%m%d')
tweets.drop(columns = ['date'], inplace = True)
tweets['date'] = date

# Reorder columns
cols = tweets.columns.tolist()
cols = cols[-1:] + cols[:-1]
tweets = tweets[cols].copy()

print('Tweet per day:')
print()
print(tweets.groupby('date').count()['id'])
# print()
# print()
# print('Minimum Tweet ID per day:')
# print()
# print(df_SanSiro.groupby('date').min('id')['id'])
# print()
# print()
# print('Maximum Tweet ID per day:')
# print()
# print(df_SanSiro.groupby('date').max('id')['id'])

Tweet per day:

date
2022-12-22    293
2022-12-23    159
2022-12-24     87
2022-12-25     44
2022-12-26     55
2022-12-27     75
2022-12-28     93
2022-12-29     44
Name: id, dtype: int64


In [17]:
# Create a DataFrame with the authors of the tweets and their respective frequency
freq_authors = tweets['author'].value_counts()
freq_authors.head()

mrmcphisto         114
lucague46760617     73
aleale2780          49
Lucapadova25        49
SipariodiPerle      29
Name: author, dtype: int64

In [18]:
# Extract all hashtags from the full text
tweets['hashtags_list'] = tweets['text'].apply(lambda x: re.findall(r"#(\w+)", x))

# Extract all mentions from the full text
tweets['mentions'] = tweets['text'].apply(lambda x: re.findall(r"@(\w+)", x))

# \w matches any single letter, number or underscore (same as [a zA Z0 9_])

#### Data Cleaning

NLTK - Natural Language ToolKit is a platform for building Python programs to work with human language data. It provides easy to use interfaces to over 50 corpora and lexical resources, along with a suite of text processing libraries for classification, tokenization, stemming, tagging, parsing, and semantic reasoning.

In [19]:
import nltk
from nltk import FreqDist
nltk.download
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Text Pre-Processing

In [20]:
def preprocess_data(data):
    # remove numbers and turning words into lower case
    data = data.astype(str).str.replace('\d+','')
    lower_text = data.str.lower()
    lemmatizer = nltk.stem.WordNetLemmatizer()
    w_tokenizer = TweetTokenizer()
    
    # token lemmatization (ex. goes --> go)
    def lemmatize_text(text):
        return[(lemmatizer.lemmatize(w)) for w in w_tokenizer.tokenize((text))]
    
    # remove punctuation
    def remove_punctuation(words):
        new_words = []
        for word in words:
            new_word = re.sub(r'[\w\s]', '', (word))
            if new_word != '':
                new_words.append(new_word)
        return new_words
    
    words = lower_text.apply(lemmatize_text)
    words = words.apply(remove_punctuation)
    return pd.DataFrame(words)

In [21]:
# apply preprocess_data function
pre_tweets = preprocess_data(tweets['text'])
tweets['text_proc'] = pre_tweets

# delete italian stopwords
stop_words = set(stopwords.words('italian'))
tweets['text_proc'] = tweets['text_proc'].apply(lambda x: [item for item in x if item not in stop_words])

C:\Users\FC\AppData\Local\Temp\ipykernel_15524\3179111472.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  data = data.astype(str).str.replace('\d+','')


## Analysis

### Social Content Analysis

#### _Sentiment Analysis_

When dealing with social media text, we usually want to identify urls, hashtags, smileys as separate objects and do not tokenize it to individual characters.

VADER

In [22]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\FC\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [23]:
sent_analyzer = SentimentIntensityAnalyzer()

In [28]:
tweets[['text', 'text_proc']]

,text,text_proc
0,RT @JackRossano: La #Meloni per la #famiglia (...,"[@, :, #, #, (, ), #, #, ://./]"
1,RT @ValePieraccini: Siamo indietro secoli luce...,"[@, :, ., #]"
2,RT @lucague46760617: #smartworking @fattoquoti...,"[@, :, #, @, …]"
3,RT @Ernesto23710724: @AlbertoBagnai Forse deve...,"[@, :, @, #, …]"
4,RT @LucaGalizia: Chi già preconizza (con mezzo...,"[@, :, (, ), #, ,, #, ,, …]"
...,...,...
845,RT @Bababirbante: Dare e togliere lo #smartwor...,"[@, :, #, …]"
846,RT @JackRossano: Il problema dello #smartworki...,"[@, :, #, ., …]"
847,RT @catesap7: Che schifo!!! Non avete idea cos...,"[@, :, !, !, !, ., …]"
848,"RT @beeconfortzone: Cara @GiorgiaMeloni , un m...","[@, :, @, ,, ,, ,, #, …]"


In [29]:
tweets['scores'] = tweets['text'].apply(lambda Tweet:sent_analyzer.polarity_scores(Tweet))
tweets['compound'] = tweets['scores'].apply(lambda score_dict:score_dict['compound'])

tweets.head()

,date,id,text,like,n_rt,author,location,retweeted,user_mentions,hastags,hashtags_list,mentions,text_proc,scores,compound
0,2022-12-29,1608467861609287680,RT @JackRossano: La #Meloni per la #famiglia (...,0,15,atlantidelibri,castel san pietro Bo,False,"[{'screen_name': 'JackRossano', 'name': 'Jack ...","[{'text': 'Meloni', 'indices': [20, 27]}, {'te...","[Meloni, famiglia, governodegliorrori, smartwo...",[JackRossano],"[@, :, #, #, (, ), #, #, ://./]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
1,2022-12-29,1608466273205374978,RT @ValePieraccini: Siamo indietro secoli luce...,0,2,saragra98354743,,False,"[{'screen_name': 'ValePieraccini', 'name': 'Va...","[{'text': 'smartworking', 'indices': [48, 61]}]",[smartworking],[ValePieraccini],"[@, :, ., #]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,2022-12-29,1608465798594695168,RT @lucague46760617: #smartworking @fattoquoti...,0,3,saragra98354743,,False,"[{'screen_name': 'lucague46760617', 'name': 'l...","[{'text': 'smartworking', 'indices': [21, 34]}]",[smartworking],"[lucague46760617, fattoquotidiano]","[@, :, #, @, …]","{'neg': 0.0, 'neu': 0.856, 'pos': 0.144, 'comp...",0.4019
3,2022-12-29,1608465280056922113,RT @Ernesto23710724: @AlbertoBagnai Forse deve...,0,1,saragra98354743,,False,"[{'screen_name': 'Ernesto23710724', 'name': 'E...","[{'text': 'smartworking', 'indices': [89, 102]}]",[smartworking],"[Ernesto23710724, AlbertoBagnai]","[@, :, @, #, …]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
4,2022-12-29,1608464910668926976,RT @LucaGalizia: Chi già preconizza (con mezzo...,0,1,NicoladelConte8,,False,"[{'screen_name': 'LucaGalizia', 'name': 'Luca ...","[{'text': 'lockdown', 'indices': [68, 77]}, {'...","[lockdown, smartworking]",[LucaGalizia],"[@, :, (, ), #, ,, #, ,, …]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000


FEEL-IT: Emotion and Sentiment Classification for the Italian Language.

https://towardsdatascience.com/sentiment-analysis-and-emotion-recognition-in-italian-using-bert-92f5c8fe8a2

In [33]:
from feel_it import EmotionClassifier, SentimentClassifier

sentiment_classifier = SentimentClassifier()
emotion_classifier = EmotionClassifier()

Downloading: 100%|██████████| 899/899 [00:00<00:00, 449kB/s]
Downloading: 100%|██████████| 794k/794k [00:00<00:00, 8.64MB/s]
Downloading: 100%|██████████| 1.68M/1.68M [00:01<00:00, 1.57MB/s]
Downloading: 100%|██████████| 414/414 [00:00<00:00, 212kB/s]
Downloading: 100%|██████████| 443M/443M [00:10<00:00, 40.3MB/s] 


The feel-it-italian-sentiment model performs sentiment analysis on Italian. We fine-tuned the UmBERTo model on our new dataset (i.e., FEEL-IT) obtaining state-of-the-art performances on different benchmark corpora.

In [50]:
print(sentiment_classifier.predict(tweets["text"].values.tolist()))

['negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'po

In [36]:
tweets_sentiment = tweets.copy()

In [40]:
text = tweets_sentiment['text']
li_sent = []
for i in range(0, text.shape[0]):
  sent = sentiment_classifier.predict([text[i]])
  li_sent.append(sent)
  if i % 5000 == 0:
    print('Riga',i,'su',text.shape[0])

tweets_sentiment['sentiment_BERT'] = [item for sublist in li_sent for item in sublist]
     
positive = []
negative = []

Riga 0 su 850


In [46]:
positive = []
negative = []
ratio = []

for line in tweets_sentiment.values:
  sent = line[15]

  if sent == 'negative':
    positive.append(0)
    negative.append(1)
    ratio.append(-1)
  else:
    positive.append(1)
    negative.append(0)
    ratio.append(1)

In [47]:
tweets_sentiment['positive'] = positive
tweets_sentiment['negative'] = negative
tweets_sentiment['ratio'] = ratio

In [48]:
tweets_sentiment.head()

,date,id,text,like,n_rt,author,location,retweeted,user_mentions,hastags,hashtags_list,mentions,text_proc,scores,compound,sentiment_BERT,positive,negative,ratio
0,2022-12-29,1608467861609287680,RT @JackRossano: La #Meloni per la #famiglia (...,0,15,atlantidelibri,castel san pietro Bo,False,"[{'screen_name': 'JackRossano', 'name': 'Jack ...","[{'text': 'Meloni', 'indices': [20, 27]}, {'te...","[Meloni, famiglia, governodegliorrori, smartwo...",[JackRossano],"[@, :, #, #, (, ), #, #, ://./]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,negative,0,1,-1
1,2022-12-29,1608466273205374978,RT @ValePieraccini: Siamo indietro secoli luce...,0,2,saragra98354743,,False,"[{'screen_name': 'ValePieraccini', 'name': 'Va...","[{'text': 'smartworking', 'indices': [48, 61]}]",[smartworking],[ValePieraccini],"[@, :, ., #]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,negative,0,1,-1
2,2022-12-29,1608465798594695168,RT @lucague46760617: #smartworking @fattoquoti...,0,3,saragra98354743,,False,"[{'screen_name': 'lucague46760617', 'name': 'l...","[{'text': 'smartworking', 'indices': [21, 34]}]",[smartworking],"[lucague46760617, fattoquotidiano]","[@, :, #, @, …]","{'neg': 0.0, 'neu': 0.856, 'pos': 0.144, 'comp...",0.4019,negative,0,1,-1
3,2022-12-29,1608465280056922113,RT @Ernesto23710724: @AlbertoBagnai Forse deve...,0,1,saragra98354743,,False,"[{'screen_name': 'Ernesto23710724', 'name': 'E...","[{'text': 'smartworking', 'indices': [89, 102]}]",[smartworking],"[Ernesto23710724, AlbertoBagnai]","[@, :, @, #, …]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,negative,0,1,-1
4,2022-12-29,1608464910668926976,RT @LucaGalizia: Chi già preconizza (con mezzo...,0,1,NicoladelConte8,,False,"[{'screen_name': 'LucaGalizia', 'name': 'Luca ...","[{'text': 'lockdown', 'indices': [68, 77]}, {'...","[lockdown, smartworking]",[LucaGalizia],"[@, :, (, ), #, ,, #, ,, …]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,negative,0,1,-1


In [49]:
tweets_sentiment.to_csv('../data/SW_sentiment.csv')

#### _Emotion Analysis_

Recognizing emotions in text is fundamental to get a better sense of how people are talking about something. People can talk about a new event, but positive/negative labels might not be enough.

The feel-it-italian-emotion model performs emotion classification (joy, fear, anger, sadness) on Italian. We fine-tuned the UmBERTo model on our new dataset (i.e., FEEL-IT) obtaining state-of-the-art performances on different benchmark corpora.

In [35]:
print(emotion_classifier.predict(tweets["text"].values.tolist()))

['anger', 'sadness', 'anger', 'anger', 'anger', 'anger', 'joy', 'anger', 'anger', 'joy', 'anger', 'anger', 'joy', 'joy', 'anger', 'joy', 'joy', 'anger', 'anger', 'sadness', 'anger', 'sadness', 'anger', 'anger', 'sadness', 'sadness', 'joy', 'sadness', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'fear', 'anger', 'anger', 'anger', 'fear', 'anger', 'sadness', 'anger', 'anger', 'anger', 'joy', 'anger', 'anger', 'anger', 'joy', 'anger', 'sadness', 'joy', 'anger', 'joy', 'anger', 'joy', 'joy', 'anger', 'anger', 'anger', 'anger', 'joy', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'anger', 'fear', 'joy', 'sadness', 'anger', 'anger', 'anger', 'anger', 'anger', 'joy', 'fear', 'joy', 'anger', 'anger', 'fear', 'fear', 'joy', 'joy', 'joy', 'joy', 'anger', 'anger', 'anger', 'anger', 'sadness', 'anger', 'anger', 'anger', 'anger', 'joy', 'joy', 'sadness', 'sadness', 'anger', 'anger', 'joy', 'fear', 'anger', 'sadness', 'anger', 'fear', 'fear', 'anger', 'anger', 'anger', 'ang

In [51]:
tweets_emotion = tweets.copy()

In [53]:
text = tweets_emotion['text']
li_emotion = []
for i in range(0, text.shape[0]):
  emotion = emotion_classifier.predict([text[i]])
  li_emotion.append(emotion)
  if i % 5000 == 0:
    print('Riga',i,'su',text.shape[0])

tweets_emotion['emotion_BERT'] = [item for sublist in li_emotion for item in sublist]
     
anger = []
joy = []
fear = []
sadness = []

Riga 0 su 850


In [56]:
anger = []
joy = []
fear = []
sadness = []

for line in tweets_emotion.values:
  emotion = line[15]

  if emotion == 'anger':
    anger.append(1)
    joy.append(0)
    fear.append(0)
    sadness.append(0)
  elif emotion == 'joy':
    anger.append(0)
    joy.append(1)
    fear.append(0)
    sadness.append(0)
  elif emotion == 'fear':
    anger.append(0)
    joy.append(0)
    fear.append(1)
    sadness.append(0)
  else:
    anger.append(0)
    joy.append(0)
    fear.append(0)
    sadness.append(1)

In [57]:
tweets_emotion['anger'] = anger
tweets_emotion['joy'] = joy
tweets_emotion['fear'] = fear
tweets_emotion['sadness'] = sadness

In [58]:
tweets_emotion.head()

,date,id,text,like,n_rt,author,location,retweeted,user_mentions,hastags,hashtags_list,mentions,text_proc,scores,compound,emotion_BERT,anger,joy,fear,sadness
0,2022-12-29,1608467861609287680,RT @JackRossano: La #Meloni per la #famiglia (...,0,15,atlantidelibri,castel san pietro Bo,False,"[{'screen_name': 'JackRossano', 'name': 'Jack ...","[{'text': 'Meloni', 'indices': [20, 27]}, {'te...","[Meloni, famiglia, governodegliorrori, smartwo...",[JackRossano],"[@, :, #, #, (, ), #, #, ://./]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,anger,1,0,0,0
1,2022-12-29,1608466273205374978,RT @ValePieraccini: Siamo indietro secoli luce...,0,2,saragra98354743,,False,"[{'screen_name': 'ValePieraccini', 'name': 'Va...","[{'text': 'smartworking', 'indices': [48, 61]}]",[smartworking],[ValePieraccini],"[@, :, ., #]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,sadness,0,0,0,1
2,2022-12-29,1608465798594695168,RT @lucague46760617: #smartworking @fattoquoti...,0,3,saragra98354743,,False,"[{'screen_name': 'lucague46760617', 'name': 'l...","[{'text': 'smartworking', 'indices': [21, 34]}]",[smartworking],"[lucague46760617, fattoquotidiano]","[@, :, #, @, …]","{'neg': 0.0, 'neu': 0.856, 'pos': 0.144, 'comp...",0.4019,anger,1,0,0,0
3,2022-12-29,1608465280056922113,RT @Ernesto23710724: @AlbertoBagnai Forse deve...,0,1,saragra98354743,,False,"[{'screen_name': 'Ernesto23710724', 'name': 'E...","[{'text': 'smartworking', 'indices': [89, 102]}]",[smartworking],"[Ernesto23710724, AlbertoBagnai]","[@, :, @, #, …]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,anger,1,0,0,0
4,2022-12-29,1608464910668926976,RT @LucaGalizia: Chi già preconizza (con mezzo...,0,1,NicoladelConte8,,False,"[{'screen_name': 'LucaGalizia', 'name': 'Luca ...","[{'text': 'lockdown', 'indices': [68, 77]}, {'...","[lockdown, smartworking]",[LucaGalizia],"[@, :, (, ), #, ,, #, ,, …]","{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,anger,1,0,0,0


In [59]:
tweets_emotion.to_csv('../data/SW_emotion.csv')

---

#### Social Network Analysis

Measures of Centrality

Community Detection

## Visualization